In [19]:
#auto update selenium
# webdriver_auto_update.py
from webdriver_auto_update import check_driver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time 
import os

#web driver folder
WEB_DRIVER_INSTALL_PATH = "drivers"
print("Checking webdriver")
#check and install Selenium web driver.
check_driver(WEB_DRIVER_INSTALL_PATH)

Checking webdriver
No chromedriver.exe found in specified path

Attempting to download latest driver online......
https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip


PermissionError: [Errno 13] Permission denied: 'drivers\\chromedriver.exe'

In [20]:
def get_selenium():                           
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('headless')                        
    driver = webdriver.Chrome(executable_path='drivers\chromedriver.exe',chrome_options=options)
    return (driver)

In [21]:
#scrape cards from CFVG site
resource_url = "https://en.cf-vanguard.com/cardlist/cardsearch/?expansion=167"

In [22]:
browser = get_selenium()                           
browser.get(resource_url)

#get length of page
lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight); var lenOfPage=document.body.scrollHeight;return lenOfPage;")
print(lenOfPage)

C:\Users\at_be\AppData\Local\Temp\ipykernel_6060\1523202300.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='drivers\chromedriver.exe',chrome_options=options)
C:\Users\at_be\AppData\Local\Temp\ipykernel_6060\1523202300.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path='drivers\chromedriver.exe',chrome_options=options)


2623


In [23]:
# Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
from bs4 import BeautifulSoup

match = False
while not match:
    last_count = lenOfPage
    time.sleep(3)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if last_count == lenOfPage:
        match = True

#load page data
source_data = browser.page_source

#grab beautiful soup object
page_soup = BeautifulSoup(source_data)





In [26]:
results = page_soup.findAll("div","cardlist_gallerylist")
print(results)

[<div class="cardlist_gallerylist" id="cardlist-container">
<ul>
<li><a href="/cardlist/?cardno=D-BT01/001EN&amp;expansion=167"><img alt="Vairina Valiente" class="object-fit-img" src="https://s3-ap-northeast-1.amazonaws.com/en.cf-vanguard.com/wordpress/wp-content/images/cardlist/dbt01/dbt01_001.png" title="Vairina Valiente"/></a></li>
<li><a href="/cardlist/?cardno=D-BT01/002EN&amp;expansion=167"><img alt="Heavy Artillery of Dust Storm, Eugene" class="object-fit-img" src="https://s3-ap-northeast-1.amazonaws.com/en.cf-vanguard.com/wordpress/wp-content/images/cardlist/dbt01/dbt01_002.png" title="Heavy Artillery of Dust Storm, Eugene"/></a></li>
<li><a href="/cardlist/?cardno=D-BT01/003EN&amp;expansion=167"><img alt="Master of Gravity, Baromagnes" class="object-fit-img" src="https://s3-ap-northeast-1.amazonaws.com/en.cf-vanguard.com/wordpress/wp-content/images/cardlist/dbt01/dbt01_003.png" title="Master of Gravity, Baromagnes"/></a></li>
<li><a href="/cardlist/?cardno=D-BT01/004EN&amp;exp